In [1]:
import numpy as np

In [2]:
# typical values for temperature, molar ratio, and pressure
T = 873.15
y0 = np.array([0.3, 0.2, 0.0, 0.0, 0.0, 0.5]) # dimensionless
P = 1 # bar
Mr = np.array([16.04e-03, 44.01e-03, 18.01528e-03, 1.00794e-03, 28.01e-03, 39.948e-03]) # kg/mol

In [4]:
# literature information contained in the NIST SOURCE Data Archive - https://webbook.nist.gov/
# CO2 + CH4 ⇌ 2CO + 2H2 - DRM reaction
def thermDRM(T):
    
    t = T/1000
    WBi = np.array([
          [25.56759,  6.096130,     4.054656,  -2.671301,  0.131021, -118.0089, 227.3665,   -110.5271],  # CO
          [-0.703029, 108.4773,    -42.52157,   5.862788,  0.678565, -76.84376, 158.7163,   -74.87310],  # CH4
          [24.99735,  55.18696,   -33.69137,    7.948387, -0.136638, -403.6075, 228.2431,   -393.5224],  # CO2
          [33.066178, -11.363417,  11.432816,  -2.772874, -0.158558, -9.980797, 172.707974,    0.0]      # H2
          ])
    
    # heats of formation at 298.15 K for 'CO', 'CH4', 'CO2', 'H2'
    Hfi_298 = np.array([-110.53, -74.85, -393.51, 0.0]) # kJ/mol
     
    Sfi_298 = np.array([197.66, 186.25, 213.79, 130.68]) # J/mol*K
    
    Stoichiometry = np.array([ 2,    # 'CO'
                              -1,    # 'CH4'
                              -1,    # 'CO2'
                               2])   # 'H2'
    
    Hrxn298 = np.dot(Hfi_298, Stoichiometry)
    Srxn298 = np.dot(Sfi_298, Stoichiometry)
    Grxn298 = Hrxn298 - 298.15*(Srxn298)/1000
    
    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
    T_Si = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])
    
    Hi = np.dot(WBi, T_Hi)        # (H - H_298.15) kJ/mol
    Si = np.dot(WBi, T_Si/1000.0) # absolute entropy kJ/mol/K
    
    Hi_stoichiometry = Hi*Stoichiometry
    Si_stoichiometry = Si*Stoichiometry
    
    sum_Hi = sum(Hi_stoichiometry)
    sum_Si = sum(Si_stoichiometry)
    
    Hrxn = Hrxn298 + sum_Hi
    Grxn = Hrxn - T*sum_Si
  
    R = 8.314e-3 # kJ/mol/K
    
    Keq = np.exp(-Grxn/R/T)
    
    return np.array([Keq, Hrxn])

Keq_DRM = thermDRM(T)[0]
Hrxn_DRM = thermDRM(T)[1]

In [5]:
# literature information contained in the NIST SOURCE Data Archive - https://webbook.nist.gov/
# CH4 + H2O ⇌ CO + 3H2 - SRM1 reaction
def thermSRM1(T): 
    
    t = T/1000            
    WBi = np.array([
          [25.56759,  6.096130,     4.054656,  -2.671301,  0.131021, -118.0089, 227.3665,   -110.5271],  # CO
          [-0.703029, 108.4773,    -42.52157,   5.862788,  0.678565, -76.84376, 158.7163,   -74.87310],  # CH4
          [30.09200,  6.832514,     6.793435,  -2.534480,  0.082139, -250.8810, 223.3967,   -241.8264],  # H2O
          [33.066178, -11.363417,  11.432816,  -2.772874, -0.158558, -9.980797, 172.707974,    0.0]      # H2
          ])
    
    
    # heats of formation at 298.15 K for 'CO', 'CH4', 'H2O', 'H2'
    Hfi_298 = np.array([-110.53, -74.85, -241.826, 0.0]) # kJ/mol
     
    Sfi_298 = np.array([197.66, 186.25, 188.84, 130.68]) # J/mol*K
    
    Stoichiometry = np.array([ 1,    # 'CO'
                              -1,    # 'CH4'
                              -1,    # 'H2O'
                               3])   # 'H2'
    
    Hrxn298 = np.dot(Hfi_298, Stoichiometry)
    Srxn298 = np.dot(Sfi_298, Stoichiometry)
    Grxn298 = Hrxn298 - 298.15*(Srxn298)/1000
    
    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
    T_Si = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])
    
    Hi = np.dot(WBi, T_Hi)        # (H - H_298.15) kJ/mol
    Si = np.dot(WBi, T_Si/1000.0) # absolute entropy kJ/mol/K
    
    Hi_stoichiometry = Hi*Stoichiometry
    Si_stoichiometry = Si*Stoichiometry
    
    sum_Hi = sum(Hi_stoichiometry)
    sum_Si = sum(Si_stoichiometry)
    
    Hrxn = Hrxn298 + sum_Hi
    Grxn = Hrxn - T*sum_Si

    R = 8.314e-3 # kJ/mol/K
    
    Keq = np.exp(-Grxn/R/T)
    return np.array([Keq, Hrxn])

Keq_SRM1 = thermSRM1(T)[0]
Hrxn_SRM1 = thermSRM1(T)[1]

In [6]:
# literature information contained in the NIST SOURCE Data Archive - https://webbook.nist.gov/
# CH4 + 2H2O ⇌ CO2 + 4H2 - SRM2 reaction
def thermSRM2(T):

    t = T/1000                  
    WBi = np.array([
          [24.99735,  55.18696,   -33.69137,    7.948387, -0.136638, -403.6075, 228.2431,   -393.5224],  # CO2
          [-0.703029, 108.4773,    -42.52157,   5.862788,  0.678565, -76.84376, 158.7163,   -74.87310],  # CH4
          [30.09200,  6.832514,     6.793435,  -2.534480,  0.082139, -250.8810, 223.3967,   -241.8264],  # H2O
          [33.066178, -11.363417,  11.432816,  -2.772874, -0.158558, -9.980797, 172.707974,    0.0]      # H2
          ])
    
    
    # heats of formation at 298.15 K for 'CO2', 'CH4', 'H2O', 'H2'
    Hfi_298 = np.array([-393.51, -74.85, -241.826, 0.0]) # kJ/mol
     
    Sfi_298 = np.array([213.79, 186.25, 188.84, 130.68]) # J/mol*K
    
    Stoichiometry = np.array([ 1,    # 'CO2'
                              -1,    # 'CH4'
                              -2,    # 'H2O'
                               4])   # 'H2'
    
    Hrxn298 = np.dot(Hfi_298, Stoichiometry)
    Srxn298 = np.dot(Sfi_298, Stoichiometry)
    Grxn298 = Hrxn298 - 298.15*(Srxn298)/1000
    
    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
    T_Si = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])
    
    Hi = np.dot(WBi, T_Hi)        # (H - H_298.15) kJ/mol
    Si = np.dot(WBi, T_Si/1000.0) # absolute entropy kJ/mol/K
    
    Hi_stoichiometry = Hi*Stoichiometry
    Si_stoichiometry = Si*Stoichiometry
    
    sum_Hi = sum(Hi_stoichiometry)
    sum_Si = sum(Si_stoichiometry)
    
    Hrxn = Hrxn298 + sum_Hi
    Grxn = Hrxn - T*sum_Si
    
    R = 8.314e-3 # kJ/mol/K
    
    Keq = np.exp(-Grxn/R/T)
    
    return np.array([Keq, Hrxn])

Keq_SRM2 = thermSRM2(T)[0]
Hrxn_SRM2 = thermSRM2(T)[1]

In [7]:
# literature information contained in the NIST SOURCE Data Archive - https://webbook.nist.gov/
# CO + H2O ⇌ CO2 + H2 - WGS reaction
def thermWGS(T):   

    t = T/1000                 
    WBi = np.array([
          [25.56759,  6.096130,     4.054656,  -2.671301,  0.131021, -118.0089, 227.3665,   -110.5271],  # CO
          [30.09200,  6.832514,     6.793435,  -2.534480,  0.082139, -250.8810, 223.3967,   -241.8264],  # H2O
          [24.99735,  55.18696,   -33.69137,    7.948387, -0.136638, -403.6075, 228.2431,   -393.5224],  # CO2
          [33.066178, -11.363417,  11.432816,  -2.772874, -0.158558, -9.980797, 172.707974,    0.0]      # H2
          ])
    
    
    # heats of formation at 298.15 K for 'CO', 'H2O', 'CO2', 'H2'
    Hfi_298 = np.array([-110.53, -241.826, -393.51, 0.0]) # kJ/mol
     
    Sfi_298 = np.array([197.66, 188.84, 213.79, 130.68]) # J/mol*K
    
    Stoichiometry = np.array([-1,    # 'CO'
                              -1,    # 'H2O'
                               1,    # 'CO2'
                               1])   # 'H2'
    
    Hrxn298 = np.dot(Hfi_298, Stoichiometry)
    Srxn298 = np.dot(Sfi_298, Stoichiometry)
    Grxn298 = Hrxn298 - 298.15*(Srxn298)/1000
    
    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
    T_Si = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])
    
    Hi = np.dot(WBi, T_Hi)        # (H - H_298.15) kJ/mol
    Si = np.dot(WBi, T_Si/1000.0) # absolute entropy kJ/mol/K
    
    Hi_stoichiometry = Hi*Stoichiometry
    Si_stoichiometry = Si*Stoichiometry
    
    sum_Hi = sum(Hi_stoichiometry)
    sum_Si = sum(Si_stoichiometry)
    
    Hrxn = Hrxn298 + sum_Hi
    Grxn = Hrxn - T*sum_Si
    
    R = 8.314e-3 # kJ/mol/K
    
    Keq = np.exp(-Grxn/R/T)

    return np.array([Keq, Hrxn])

Keq_WGS = thermWGS(T)[0]
Hrxn_WGS = thermWGS(T)[1]

In [8]:
def ideal_gas(T, P, y):
    
    C = y*P/(8.314e-05*T) # mol/m3

    return np.array(C)

Ci = ideal_gas(T, P, y0)

Cto = ideal_gas(T, P, y0).sum()

d_mix = ideal_gas(T, P, y0).sum()*(Mr*y0).sum() # kg/m3

In [9]:
# Mr of mixture in kg/mol
def Mr_mixture(y):
    
    return (Mr*y).sum()

Mr_mix = Mr_mixture(y0)
Mr_mix

0.033588

In [10]:
# specific heat
def heat_cap(T, y):
    
    WBi = np.array([
      [-0.703029, 108.4773,     -42.52157,       5.862788,       0.678565],       # CH4  
      [24.99735,   55.18696,    -33.69137,       7.948387,      -0.136638],       # CO2
      [30.09200,    6.832514,     6.793435,     -2.534480,       0.082139],       # H2O
      [25.56759,    6.096130,     4.054656,     -2.671301,       0.131021],       # CO
      [33.066178, -11.363417,    11.432816,     -2.772874,      -0.158558],       # H2    
      [20.78600,    2.825911e-07, -1.464191e-07,  1.092131e-08, -3.661371e-08],   # Ar
      ])
    
    t = T/1000
    
    T_Cpi = np.array([1,  t, t**2, t**3, 1.0/t])
    
    # specific heat capacities of species in J/mol/K for p = 1 bar
    Cpi = np.dot(WBi, T_Cpi)

    return np.array(Cpi)

# which is an array of objects
Cpi = heat_cap(T, y0)
Cp_mix_mole = (np.dot(Cpi, y0)).sum()
Cp_mix_mass = np.dot(heat_cap(T, y0), y0).sum()/Mr_mix

In [11]:
# partial enthalpies
def Hi(T, y):
    
    t = T/1000
    
    WBi = np.array([
      [-0.703029, 108.4773,      -42.52157,       5.862788,      0.678565,    -76.84376,  158.7163,   -74.87310],  # CH4  
      [24.99735,   55.18696,     -33.69137,       7.948387,     -0.136638,   -403.6075,   228.2431,  -393.5224],   # CO2
      [30.09200,    6.832514,      6.793435,     -2.534480,      0.082139,   -250.8810,   223.3967,  -241.8264],   # H2O
      [33.066178, -11.363417,     11.432816,     -2.772874,     -0.158558,     -9.980797, 172.707974,   0.00000],  # H2
      [25.56759,    6.096130,      4.054656,     -2.671301,      0.131021,   -118.0089,   227.3665,  -110.52710],  # CO    
      [20.78600,    2.825911e-07, -1.464191e-07,  1.092131e-08, -3.661371e-08, -6.197350, 179.9990,     0.00000],  # Ar
      ])

    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
    
    Hi = WBi@T_Hi # (H - H_298.15) kJ/mol
    
    # heats of formation at 298.15 K for 'CH4', 'CO2', 'H2O', 'H2', 'CO', 'Ar' in kJ/mol
    Hfi_298 = np.array([-74.85, -393.51, -241.826,  0.0, -110.53, 0.0])
    
    return (Hi + Hfi_298)*1000 # J/mol

Hi(T, y0)

array([ -45443.95578899, -366896.08539025, -220956.18251254,
         16874.79203257,  -93002.16060057,   11951.94601875])